## Todo List
- [x] Checar se formato de datas conferem.
- [x] Remover UGs sem liberação OC
- [x] Considerar coluna montagem. Onde estiver vazio, pegar valor de Torres para EOL, no skate.
- [ ] Separar um cojunto aleatório para teste
- [x] Calcular tempos de construção por marco: (IO_realizado - realizado), (CC_realizado - realizado) ...etc  (ordem:IO,CC,ME,OT)
- [x] Calcular atraso da operação comercial (realizado - outorgado)
- [x] Calcular média e dp do tempo de construção.
- [x] Calcular data dos marcos medios (outorgado - tempo_medio_de_construcao) por fonte.
- [x] Calcular atrasos com base nos marcos medios
- [ ] Calcular media e dp dos atrasos. 
- [ ] Normalizar os atrasos calculados (atraso - atraso_medio)/dp
- [ ] Construir matriz de correlação de Pearson
- [ ] Calcular os coeficientes para regressão linear
- [x] Para UHE pegar todas UGs, para as demais pegar a primeira UG apenas 
- [ ] Comparar pegando primeira UG e media por grupo



In [1]:
import pandas as pd
import numpy as np
import re
import pyodbc
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats
from itertools import combinations

In [2]:
def outliers(df:pd.DataFrame,cols : list):
    index_outliers =  []
    for col in cols:
        s = df[col]
        Q1 = s.quantile(0.1)
        Q3 = s.quantile(0.9)
        lower_bound =  Q1
        upper_bound =  Q3 
        index_outliers.extend(s.index[(s < lower_bound) | (s > upper_bound)])
    return set(index_outliers)

In [3]:
bd_combined = pd.read_parquet("bd_combined.gzip")
bd_combined.index.name = "idx"
bd_combined.IdeUsinaOutorga = bd_combined.IdeUsinaOutorga.astype(str)
mask = (bd_combined.SigTipoGeracao == "UHE") & (bd_combined.IdeUsinaOutorga.duplicated(keep=False))
bd_combined.loc[mask,"IdeUsinaOutorga"] = bd_combined.loc[mask,"IdeUsinaOutorga"] +"_"+ bd_combined.loc[mask].groupby("IdeUsinaOutorga").cumcount().astype(str)

In [4]:
bd_combined = bd_combined[bd_combined.DatUGInicioOpComerOutorgado.notna()
                            & bd_combined.DatLiberOpComerRealizado.notna()].copy(deep=True)

In [5]:
nomes_tempo_construcao = {
    "IO_OC" : "DatInicioObraRealizado",
    "CC_OC" : "DatConcretagemRealizado",
    "ME_OC" : "DatMontagemRealizado",
    "MT_OC" : "DatConclusaoTorresRealizado",
    "OT_OC" : "DatLiberacaoSFGTeste"
} 

tempo_construcao_por_UG = bd_combined[["IdeUsinaOutorga","SigTipoGeracao"]].copy(deep=True)
for nome,marco in nomes_tempo_construcao.items():
    tempo_construcao_por_UG[nome] = bd_combined["DatLiberOpComerRealizado"]-  bd_combined[marco]

tempo_construcao_por_UG["Atraso"]  =  bd_combined["DatLiberOpComerRealizado"]-  bd_combined["DatUGInicioOpComerOutorgado"] 
tempo_construcao_por_UG.sample(10)

,IdeUsinaOutorga,SigTipoGeracao,IO_OC,CC_OC,ME_OC,MT_OC,OT_OC,Atraso
idx,,,,,,,,
941,30337,PCH,NaT,NaT,NaT,NaT,17 days,0 days
3754,36985,EOL,247 days,191 days,60 days,31 days,44 days,-241 days
4262,38310,EOL,319 days,288 days,129 days,36 days,41 days,-762 days
282,28753,PCH,NaT,291 days,138 days,NaT,52 days,77 days
2170,34376,UTE,NaT,NaT,NaT,NaT,1 days,173 days
4310,38322,EOL,361 days,320 days,91 days,-32 days,62 days,-853 days
3321,34634,EOL,595 days,435 days,197 days,75 days,36 days,-122 days
111,27709,UTE,736 days,NaT,NaT,NaT,69 days,539 days
316,28819,PCH,1222 days,1222 days,603 days,NaT,32 days,2104 days


In [6]:
is_first = True 
combinacoes = list(combinations([x for x in tempo_construcao_por_UG.columns if x not in ["IdeUsinaOutorga","SigTipoGeracao","Atraso"]],2))
#combinacoes = list(combinations(["IO","CC","ME","MT","OT"],2))

for p_marco, s_marco in combinacoes:
    idx_loop =  ((tempo_construcao_por_UG[p_marco] >= tempo_construcao_por_UG[s_marco]) | 
                 tempo_construcao_por_UG[p_marco].isna() | 
                 tempo_construcao_por_UG[s_marco].isna())

    idx = idx_loop if is_first else (idx & idx_loop)
    is_first = False 

idx_usinas = idx & (tempo_construcao_por_UG.IO_OC.notna())
tempo_construcao_por_UG_filtered = tempo_construcao_por_UG[idx_usinas].copy(deep=True)
tempo_construcao_por_UG_filtered

,IdeUsinaOutorga,SigTipoGeracao,IO_OC,CC_OC,ME_OC,MT_OC,OT_OC,Atraso
idx,,,,,,,,
0,12_0,UHE,1532 days,855 days,346 days,NaT,8 days,665 days
1,12_1,UHE,1609 days,932 days,402 days,NaT,12 days,192 days
2,19,UTE,581 days,NaT,NaT,NaT,6 days,2299 days
3,42_0,UHE,1521 days,NaT,NaT,NaT,10 days,608 days
4,42_1,UHE,1553 days,NaT,NaT,NaT,11 days,0 days
...,...,...,...,...,...,...,...,...
5752,50872,UTE,98 days,NaT,98 days,NaT,41 days,-920 days
5776,51444,UTE,99 days,NaT,99 days,NaT,22 days,-385 days
5782,51450,UTE,104 days,NaT,104 days,NaT,26 days,-380 days


In [7]:
# Atrasos por usinas são calculados com base UG com menor atraso.
idx_primeira_ug = tempo_construcao_por_UG_filtered.groupby(["IdeUsinaOutorga"],sort=False).IO_OC.idxmin()
tempo_construcao_por_usina = tempo_construcao_por_UG_filtered.loc[idx_primeira_ug]
#tempo_construcao_por_usina.set_index(keys = "IdeUsinaOutorga",inplace=True)
tempo_construcao_por_usina

,IdeUsinaOutorga,SigTipoGeracao,IO_OC,CC_OC,ME_OC,MT_OC,OT_OC,Atraso
idx,,,,,,,,
0,12_0,UHE,1532 days,855 days,346 days,NaT,8 days,665 days
1,12_1,UHE,1609 days,932 days,402 days,NaT,12 days,192 days
2,19,UTE,581 days,NaT,NaT,NaT,6 days,2299 days
3,42_0,UHE,1521 days,NaT,NaT,NaT,10 days,608 days
4,42_1,UHE,1553 days,NaT,NaT,NaT,11 days,0 days
...,...,...,...,...,...,...,...,...
5752,50872,UTE,98 days,NaT,98 days,NaT,41 days,-920 days
5776,51444,UTE,99 days,NaT,99 days,NaT,22 days,-385 days
5782,51450,UTE,104 days,NaT,104 days,NaT,26 days,-380 days


In [8]:
# Cria tabela para calculos por geracao
tempo_construcao_por_geracao = pd.DataFrame()
tempo_construcao_por_geracao["SigTipoGeracao"] = bd_combined.SigTipoGeracao.unique() 


# Realiza calculos por geracao
for coluna in tempo_construcao_por_usina.columns:
    if coluna not in ["IdeUsinaOutorga","SigTipoGeracao"]:
        tempo_construcao_por_geracao[f"{coluna}_media"] = tempo_construcao_por_geracao.SigTipoGeracao.apply(lambda x : tempo_construcao_por_usina[tempo_construcao_por_usina.SigTipoGeracao==x][coluna].mean())
        tempo_construcao_por_geracao[f"{coluna}_std"] = tempo_construcao_por_geracao.SigTipoGeracao.apply(lambda x : tempo_construcao_por_usina[tempo_construcao_por_usina.SigTipoGeracao==x][coluna].std())

tempo_construcao_por_geracao.set_index("SigTipoGeracao",inplace=True)
display(tempo_construcao_por_geracao)

,IO_OC_media,IO_OC_std,CC_OC_media,CC_OC_std,ME_OC_media,ME_OC_std,MT_OC_media,MT_OC_std,OT_OC_media,OT_OC_std,Atraso_media,Atraso_std
SigTipoGeracao,,,,,,,,,,,,
UHE,1845 days 12:18:46.956521728,613 days 22:04:08.961484144,1627 days 00:43:25.025125632,604 days 14:30:26.179342560,1398 days 06:01:46.403940880,678 days 11:30:29.726488,NaT,NaT,50 days 17:56:48.849557522,64 days 21:26:09.395516380,193 days 06:34:26.086956522,531 days 07:33:41.201900560
UTE,679 days 08:01:14.035989720,761 days 05:45:52.317892152,NaT,NaT,517 days 02:32:18.461538464,774 days 21:07:44.989846400,NaT,NaT,106 days 07:03:04.366576820,156 days 03:00:44.268025456,290 days 11:54:26.838046272,674 days 08:34:28.942746432
PCH,876 days 02:01:03.716814160,507 days 23:43:17.432072712,646 days 20:44:16.310679608,478 days 18:21:05.859388440,400 days 11:06:47.881773400,283 days 01:00:47.064353672,NaT,NaT,65 days 01:16:27.610619469,83 days 05:12:26.072418782,1014 days 05:56:48.849557520,1056 days 05:06:05.083292864
CGH,895 days 13:55:12,552 days 23:12:08.916062312,666 days 22:28:05.106382976,466 days 09:12:43.861994504,481 days 15:04:11.162790696,389 days 18:02:31.538148436,NaT,NaT,115 days 15:40:24.489795918,127 days 23:51:41.487488832,494 days 13:55:12,647 days 18:53:00.888534936
EOL,525 days 05:55:44.162436552,343 days 05:27:23.850092204,456 days 17:02:57.853492336,341 days 22:02:02.615683164,281 days 14:18:04.931506848,324 days 06:05:56.541976552,242 days 19:27:38.143607704,278 days 05:03:08.468332528,54 days 14:22:32.284263959,38 days 02:30:12.920986305,22 days 00:51:10.050761421,572 days 15:18:44.174055760
UFV,376 days 04:07:56.821192052,119 days 04:37:59.635819074,NaT,NaT,244 days 07:47:17.086092716,111 days 03:01:54.695461352,NaT,NaT,71 days 01:44:54.039735099,59 days 04:40:52.840584634,-3 days +22:05:33.774834438,290 days 17:43:59.821206096


In [9]:
bd_combined_usinas = bd_combined.loc[idx_primeira_ug,:]

In [10]:
marco_medio_usina = tempo_construcao_por_usina.loc[:,["IdeUsinaOutorga"]]
def id_marco(ID,nome,marco):
    geracao = bd_combined_usinas.loc[bd_combined_usinas.IdeUsinaOutorga==ID,"SigTipoGeracao"].values[0]
    tempo_medio = tempo_construcao_por_geracao.loc[geracao,f"{nome}_media"]
    data_OC = bd_combined_usinas.loc[bd_combined_usinas.IdeUsinaOutorga==ID,"DatUGInicioOpComerOutorgado"].values[0]
    data_marco_medio = data_OC - tempo_medio
    return data_marco_medio


for nome,marco in nomes_tempo_construcao.items():
    marco_medio_usina[f"{marco}_estimado"] = marco_medio_usina.IdeUsinaOutorga.apply(lambda ID :  id_marco(ID,nome,marco))

In [11]:
marco_medio_usina

,IdeUsinaOutorga,DatInicioObraRealizado_estimado,DatConcretagemRealizado_estimado,DatMontagemRealizado_estimado,DatConclusaoTorresRealizado_estimado,DatLiberacaoSFGTeste_estimado
idx,,,,,,
0,12_0,2002-02-09 11:41:13.043478272,2002-09-15 23:16:34.974874368,2003-05-02 17:58:13.596059120,NaT,2007-01-09 06:03:11.150442478
1,12_1,2003-08-13 11:41:13.043478272,2004-03-18 23:16:34.974874368,2004-11-02 17:58:13.596059120,NaT,2008-07-12 06:03:11.150442478
2,19,2000-02-21 15:58:45.964010280,NaT,2000-08-01 21:27:41.538461536,NaT,2001-09-16 16:56:55.633423180
3,42_0,1998-11-10 11:41:13.043478272,1999-06-16 23:16:34.974874368,2000-01-31 17:58:13.596059120,NaT,2003-10-10 06:03:11.150442478
4,42_1,2000-08-11 11:41:13.043478272,2001-03-17 23:16:34.974874368,2001-11-01 17:58:13.596059120,NaT,2005-07-11 06:03:11.150442478
...,...,...,...,...,...,...
5752,50872,2022-07-17 15:58:45.964010280,NaT,2022-12-26 21:27:41.538461536,NaT,2024-02-10 16:56:55.633423180
5776,51444,2021-05-21 15:58:45.964010280,NaT,2021-10-30 21:27:41.538461536,NaT,2022-12-15 16:56:55.633423180
5782,51450,2021-05-21 15:58:45.964010280,NaT,2021-10-30 21:27:41.538461536,NaT,2022-12-15 16:56:55.633423180


In [12]:
atraso_usina = tempo_construcao_por_usina.loc[:,["IdeUsinaOutorga","SigTipoGeracao","Atraso"]]
for nome,marco in nomes_tempo_construcao.items():
    atraso_usina[f"{marco}_atraso"] = bd_combined_usinas[marco] - marco_medio_usina[f"{marco}_estimado"]

In [13]:
atraso_usina.sample(10)

,IdeUsinaOutorga,SigTipoGeracao,Atraso,DatInicioObraRealizado_atraso,DatConcretagemRealizado_atraso,DatMontagemRealizado_atraso,DatConclusaoTorresRealizado_atraso,DatLiberacaoSFGTeste_atraso
idx,,,,,,,,
733,29736_25,UHE,-649 days,-345 days +12:18:46.956521728,-532 days +00:43:25.025125632,-671 days +06:01:46.403940880,NaT,-675 days +17:56:48.849557522
400,29048,PCH,1804 days,2003 days 02:01:03.716814160,NaT,NaT,NaT,1822 days 01:16:27.610619469
2913,32955,UTE,109 days,-432 days +08:01:14.035989720,NaT,-350 days +02:32:18.461538464,NaT,-14 days +07:03:04.366576820
3798,37076,EOL,25 days,164 days 05:55:44.162436552,224 days 17:02:57.853492336,221 days 14:18:04.931506848,216 days 19:27:38.143607704,45 days 14:22:32.284263959
4012,37699,UTE,281 days,391 days 08:01:14.035989720,NaT,464 days 02:32:18.461538464,NaT,249 days 07:03:04.366576820
4054,37735,UTE,465 days,586 days 08:01:14.035989720,NaT,424 days 02:32:18.461538464,NaT,460 days 07:03:04.366576820
2823,32535,EOL,-747 days,-710 days +05:55:44.162436552,-712 days +17:02:57.853492336,-588 days +14:18:04.931506848,-525 days +19:27:38.143607704,-707 days +14:22:32.284263959
547,29458_0,UHE,887 days,444 days 12:18:46.956521728,867 days 00:43:25.025125632,673 days 06:01:46.403940880,NaT,902 days 17:56:48.849557522
1520,31412,EOL,-24 days,23 days 05:55:44.162436552,180 days 17:02:57.853492336,222 days 14:18:04.931506848,183 days 19:27:38.143607704,3 days 14:22:32.284263959


In [14]:
plot_created = False

for k,origem in enumerate(["skate","bdworgs","Ambos"]):





    if not plot_created:    
        num_subplots = 6*(len(colunas_atrasos_medio)-1)
        fig, axes = plt.subplots(num_subplots,3,figsize=(24,num_subplots*5))
        plot_created =True
    count = -1
    for i,SigTipoGeracao in enumerate(tempo_construcao_por_geracao.SigTipoGeracao):
        for nome_coluna in colunas_atrasos_medio:
            if nome_coluna == "Atraso_medio":
                continue
            count+=1
            try :
                ordered_notna = tempo_construcao_por_usina[(tempo_construcao_por_usina[nome_coluna].notna())
                                                 & (tempo_construcao_por_usina.SigTipoGeracao == SigTipoGeracao) ].sort_values(by = nome_coluna)
                x = ordered_notna[nome_coluna].dt.days.to_numpy().reshape(-1, 1)/30
                y = ordered_notna["Atraso_medio"].dt.days.to_numpy().reshape(-1, 1)/30
            
                res = stats.linregress(x.flatten(), y.flatten())
                a, b, r, p_value, std_err = res
                ax = axes[count,k]
                ax.scatter(x,y,color="blue")
                ax.set_xlabel(f"{nome_coluna} (meses)")
                ax.set_ylabel(f"Atraso_medio (meses)")
                ax.plot(x,a*x+b+1,color="red",label= f"{a:.1f}x+{b:.1f}")
                ax.set_title(f"\nTipo de geração: {SigTipoGeracao}   -  $R^2$ = {100*r**2:.1f}%\n Número de usinas: {x.shape[0]} \nOrigem: {origem}")
                ax.legend(loc="lower right")
                ax.grid()
            except (ValueError,AttributeError) as error:
                print(rf"Não foi possível calcular: {nome_coluna} para {SigTipoGeracao}")

plt.tight_layout()
plt.savefig("calculos_atrasos.pdf")

NameError: name 'colunas_atrasos_medio' is not defined